In [1]:
%pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [2]:
"""


api link:       https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguegamefinder.md
                https://github.com/swar/nba_api/tree/master/docs/nba_api
code base link: https://github.com/liannewriting/data-science-project-from-scratch-youtube-nba/blob/main/main.py
                https://youtu.be/cSnM2PHVqYs

"""

'\n\n\napi link:       https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguegamefinder.md\n                https://github.com/swar/nba_api/tree/master/docs/nba_api\ncode base link: https://github.com/liannewriting/data-science-project-from-scratch-youtube-nba/blob/main/main.py\n                https://youtu.be/cSnM2PHVqYs\n\n'

In [ ]:
# import libraries

from nba_api.stats.endpoints import leaguegamefinder
import datetime as dt
import pandas as pd
import numpy as np

In [6]:
league_start= dt.date(2022,10,17)   #league start
league_end= dt.date.today()

# to get a stupid MM/DD/YYYY format to push the parameter through
league_start_str = league_start.strftime("%m/%d/%Y")
league_end_str = league_end.strftime("%m/%d/%Y")




In [8]:
gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable=league_start_str, # start date
                                               date_to_nullable=league_end_str,     # end date
                                               league_id_nullable='00',
                                               )
# would be for specific date: date_from_nullable=game_date_str
games = gamefinder.get_data_frames()[0]
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612747,LAL,Los Angeles Lakers,0022200037,2022-10-23,LAL vs. POR,L,241,104,...,0.800,10,30,40,26,9,9,13,24,-2.0
1,22022,1610612757,POR,Portland Trail Blazers,0022200037,2022-10-23,POR @ LAL,W,239,106,...,0.815,13,39,52,19,6,5,19,20,2.0
2,22022,1610612744,GSW,Golden State Warriors,0022200042,2022-10-23,GSW vs. SAC,W,240,130,...,0.722,6,31,37,24,7,5,13,21,5.0
3,22022,1610612758,SAC,Sacramento Kings,0022200042,2022-10-23,SAC @ GSW,L,240,125,...,0.800,13,38,51,31,10,2,16,29,-5.0
4,22022,1610612764,WAS,Washington Wizards,0022200039,2022-10-23,WAS @ CLE,L,265,107,...,0.600,4,31,35,25,9,4,18,23,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,22022,1610612750,MIN,Minnesota Timberwolves,0022200010,2022-10-19,MIN vs. OKC,W,241,115,...,0.818,14,41,55,25,6,5,14,21,7.0
82,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,...,0.739,11,37,48,31,11,4,18,23,14.0
83,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,...,0.760,9,39,48,23,12,4,21,18,-14.0
84,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,...,0.857,4,27,31,16,8,3,14,25,-9.0


In [ ]:
games.columns

In [9]:
# Add additional information in separate columns from MATCHDAY

# Add a column about an opponent - always in matchup

opponent = games['MATCHUP'].str[-3:]
games['OPPONENT'] = opponent

# Is it a home or away game? > first set all to home then change those having @ to away
games['IS_HOME'] = 1
games.loc[games['MATCHUP'].str.contains('@'), 'IS_HOME'] = 0

# Add actual countable Wins
games['IS_WIN'] = 1
games.loc[games['WL'] == 'L', 'IS_WIN'] = 0

In [10]:
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,OPPONENT,IS_HOME,IS_WIN
0,22022,1610612747,LAL,Los Angeles Lakers,0022200037,2022-10-23,LAL vs. POR,L,241,104,...,40,26,9,9,13,24,-2.0,POR,1,0
1,22022,1610612757,POR,Portland Trail Blazers,0022200037,2022-10-23,POR @ LAL,W,239,106,...,52,19,6,5,19,20,2.0,LAL,0,1
2,22022,1610612744,GSW,Golden State Warriors,0022200042,2022-10-23,GSW vs. SAC,W,240,130,...,37,24,7,5,13,21,5.0,SAC,1,1
3,22022,1610612758,SAC,Sacramento Kings,0022200042,2022-10-23,SAC @ GSW,L,240,125,...,51,31,10,2,16,29,-5.0,GSW,0,0
4,22022,1610612764,WAS,Washington Wizards,0022200039,2022-10-23,WAS @ CLE,L,265,107,...,35,25,9,4,18,23,-10.0,CLE,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,22022,1610612750,MIN,Minnesota Timberwolves,0022200010,2022-10-19,MIN vs. OKC,W,241,115,...,55,25,6,5,14,21,7.0,OKC,1,1
82,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,...,48,31,11,4,18,23,14.0,LAL,1,1
83,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,...,48,23,12,4,21,18,-14.0,GSW,0,0
84,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,...,31,16,8,3,14,25,-9.0,BOS,0,0


In [11]:
teams = games.drop_duplicates(['TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME'])[['TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME']]
teams

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME
0,1610612747,LAL,Los Angeles Lakers
1,1610612757,POR,Portland Trail Blazers
2,1610612744,GSW,Golden State Warriors
3,1610612758,SAC,Sacramento Kings
4,1610612764,WAS,Washington Wizards
5,1610612739,CLE,Cleveland Cavaliers
6,1610612766,CHA,Charlotte Hornets
7,1610612737,ATL,Atlanta Hawks
8,1610612756,PHX,Phoenix Suns
9,1610612746,LAC,LA Clippers


In [12]:
# Actually import the team selection (probably I want to move this the very start later on)
# I used the drop duplicates to match it, could pull from static api
url ='https://raw.githubusercontent.com/vedranbrozovic/NBA_fantasy-tams/main/Team_selection.txt'
team_selection = pd.read_csv(url,sep='\t')
team_selection


,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,TEAM_OWNER,TEAM_PICK
0,1610612745,HOU,Houston Rockets,Miran,10
1,1610612761,TOR,Toronto Raptors,Miran,2
2,1610612763,MEM,Memphis Grizzlies,Miran,3
3,1610612765,DET,Detroit Pistons,Miran,8
4,1610612755,PHI,Philadelphia 76ers,Miran,1
5,1610612754,IND,Indiana Pacers,Miran,9
6,1610612737,ATL,Atlanta Hawks,Miran,6
7,1610612750,MIN,Minnesota Timberwolves,Miran,5
8,1610612756,PHX,Phoenix Suns,Miran,4
9,1610612752,NYK,New York Knicks,Miran,7


In [13]:
# Add info on the team owner & pick number to the games dataframe
team_selection_df = team_selection[['TEAM_ID','TEAM_OWNER','TEAM_PICK']]

games = pd.merge(games, team_selection_df, on=["TEAM_ID"])



In [14]:
# Add info on the owner of opposing team
opp_selection = team_selection[['TEAM_OWNER','TEAM_ABBREVIATION']]
opp_selection.columns = ['OPPOSING_OWNER','OPPONENT']
games = pd.merge(games, opp_selection ,on=["OPPONENT"])
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,BLK,TOV,PF,PLUS_MINUS,OPPONENT,IS_HOME,IS_WIN,TEAM_OWNER,TEAM_PICK,OPPOSING_OWNER
0,22022,1610612747,LAL,Los Angeles Lakers,0022200037,2022-10-23,LAL vs. POR,L,241,104,...,9,13,24,-2.0,POR,1,0,Vedran,6,Vedran
1,22022,1610612758,SAC,Sacramento Kings,0022200014,2022-10-19,SAC vs. POR,L,241,108,...,5,15,25,-7.0,POR,1,0,Tata,10,Vedran
2,22022,1610612756,PHX,Phoenix Suns,0022200027,2022-10-21,PHX @ POR,L,266,111,...,1,16,27,-2.0,POR,0,0,Miran,4,Vedran
3,22022,1610612747,LAL,Los Angeles Lakers,0022200016,2022-10-20,LAL vs. LAC,L,241,97,...,5,10,24,-6.0,LAC,1,0,Vedran,6,Tata
4,22022,1610612758,SAC,Sacramento Kings,0022200036,2022-10-22,SAC vs. LAC,L,239,109,...,5,12,21,-2.0,LAC,1,0,Tata,10,Tata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,22022,1610612741,CHI,Chicago Bulls,0022200007,2022-10-19,CHI @ MIA,W,240,116,...,6,14,26,8.0,MIA,0,1,Vedran,7,Tata
82,22022,1610612738,BOS,Boston Celtics,0022200022,2022-10-21,BOS @ MIA,W,241,111,...,11,19,23,7.0,MIA,0,1,Vedran,2,Tata
83,22022,1610612754,IND,Indiana Pacers,0022200029,2022-10-22,IND vs. DET,W,240,124,...,15,11,21,9.0,DET,1,1,Miran,9,Miran
84,22022,1610612753,ORL,Orlando Magic,0022200003,2022-10-19,ORL @ DET,L,240,109,...,5,18,23,-4.0,DET,0,0,Tata,6,Miran


In [15]:
# Add column to know if the game is against own team
games['IS_NOT_AGAINST_OWN'] = 1
games.loc[(games['TEAM_OWNER'] == games['OPPOSING_OWNER']),'IS_NOT_AGAINST_OWN']=0

games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,TOV,PF,PLUS_MINUS,OPPONENT,IS_HOME,IS_WIN,TEAM_OWNER,TEAM_PICK,OPPOSING_OWNER,IS_NOT_AGAINST_OWN
0,22022,1610612747,LAL,Los Angeles Lakers,0022200037,2022-10-23,LAL vs. POR,L,241,104,...,13,24,-2.0,POR,1,0,Vedran,6,Vedran,0
1,22022,1610612758,SAC,Sacramento Kings,0022200014,2022-10-19,SAC vs. POR,L,241,108,...,15,25,-7.0,POR,1,0,Tata,10,Vedran,1
2,22022,1610612756,PHX,Phoenix Suns,0022200027,2022-10-21,PHX @ POR,L,266,111,...,16,27,-2.0,POR,0,0,Miran,4,Vedran,1
3,22022,1610612747,LAL,Los Angeles Lakers,0022200016,2022-10-20,LAL vs. LAC,L,241,97,...,10,24,-6.0,LAC,1,0,Vedran,6,Tata,1
4,22022,1610612758,SAC,Sacramento Kings,0022200036,2022-10-22,SAC vs. LAC,L,239,109,...,12,21,-2.0,LAC,1,0,Tata,10,Tata,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,22022,1610612741,CHI,Chicago Bulls,0022200007,2022-10-19,CHI @ MIA,W,240,116,...,14,26,8.0,MIA,0,1,Vedran,7,Tata,1
82,22022,1610612738,BOS,Boston Celtics,0022200022,2022-10-21,BOS @ MIA,W,241,111,...,19,23,7.0,MIA,0,1,Vedran,2,Tata,1
83,22022,1610612754,IND,Indiana Pacers,0022200029,2022-10-22,IND vs. DET,W,240,124,...,11,21,9.0,DET,1,1,Miran,9,Miran,0
84,22022,1610612753,ORL,Orlando Magic,0022200003,2022-10-19,ORL @ DET,L,240,109,...,18,23,-4.0,DET,0,0,Tata,6,Miran,1


In [16]:
# actually calculate the points!
# Doesn't address the Playoff game / Championship

bodovi = games.groupby(by=['TEAM_OWNER'])['IS_WIN'].sum()
bodovi = bodovi.sort_values(ascending=False)

print("Ovo je trenutno stanje s bodovima na daatum " + league_end_str)
print(bodovi)
# bodovi = bodovi.sort_values(by=['IS_WIN'], ascending=False)



Ovo je trenutno stanje s bodovima na daatum 10/23/2022
TEAM_OWNER
Vedran    19
Miran     12
Tata      12
Name: IS_WIN, dtype: int64


In [ ]:
 # Next steps: Play with data
# Total possible points, total games, total games vs. own teams, total ratios for all pairs...

In [17]:
print("Total number of games played:")

ukupno_tekma = games.groupby(by=['TEAM_OWNER'])['GAME_ID'].count()
ukupno_tekma = ukupno_tekma.sort_values(ascending=False)

ukupno_tekma

Total number of games played:


TEAM_OWNER
Miran     29
Tata      29
Vedran    28
Name: GAME_ID, dtype: int64

In [18]:
print("Total possible points (excl. games vs. own teams):")

potencijalno_bodovi = games.groupby(by=['TEAM_OWNER'])['IS_NOT_AGAINST_OWN'].sum()
potencijalno_bodovi  = potencijalno_bodovi.sort_values(ascending=False)

potencijalno_bodovi

Total possible points (excl. games vs. own teams):


TEAM_OWNER
Tata      23
Vedran    22
Miran     19
Name: IS_NOT_AGAINST_OWN, dtype: int64

In [23]:
# Print the results


print("Ovo je trenutno stanje s bodovima na daatum " + league_end_str)
print(bodovi)
print()

print("Total number of games played:")
print(ukupno_tekma)
print()

print("Total possible points (excl. games vs. own teams):")
print(potencijalno_bodovi)

Ovo je trenutno stanje s bodovima na daatum 10/23/2022
TEAM_OWNER
Vedran    19
Miran     12
Tata      12
Name: IS_WIN, dtype: int64

Total number of games played:
TEAM_OWNER
Miran     29
Tata      29
Vedran    28
Name: GAME_ID, dtype: int64

Total possible points (excl. games vs. own teams):
TEAM_OWNER
Tata      23
Vedran    22
Miran     19
Name: IS_NOT_AGAINST_OWN, dtype: int64
